In [3]:
import torch
from torch.nn import MSELoss # class

In [4]:
# 预测标签
yhat = torch.randn(size=(50,),dtype=torch.float32)
# 真实标签
y = torch.randn(size=(50,),dtype=torch.float32)

In [5]:
criterion / loss

NameError: name 'criterion' is not defined

In [6]:
criterion = MSELoss() # 实例化类
# MSELoss(reduction = "mean均值"/"sum加和")  MSE/SSE

In [8]:
loss = criterion(yhat,y) # shift+Tab查看函数参数

In [9]:
loss

tensor(2.2788)

In [10]:
# 二分类交叉熵损失函数 对数函数 -lnP
# Loss = -(y*ln(sigma)+(1-y)*ln(1-sigma))
# y:真实标签
# sigma: 概率 sigmoid(z)
# z=Xw
# 假设X,w
# 假设总样本量 m

In [11]:
m = 3*pow(10,3)

In [15]:
torch.random.manual_seed(420) #固定随机数编号，使得每次初始化随机数固定
X = torch.rand((m,4),dtype=torch.float32)
w = torch.rand((4,1),dtype=torch.float32)
y = torch.randint(low=0,high=2,size=(m,1),dtype=torch.float32) #分类标签有时候不接受浮点类型，这里是训练使用

In [16]:
zhat = torch.mm(X,w)

In [17]:
sigma = torch.sigmoid(zhat)

In [20]:
sigma,sigma.shape

(tensor([[0.7353],
         [0.7354],
         [0.6014],
         ...,
         [0.6787],
         [0.6354],
         [0.7088]]),
 torch.Size([3000, 1]))

In [21]:
# Loss=-(y*ln(sigma)+(1-y)*ln(1-sigma))

In [25]:
loss_i = -(y*torch.log(sigma) + (1-y)*torch.log(1-sigma))

In [26]:
loss_i,loss_i.shape

(tensor([[0.3075],
         [0.3073],
         [0.9198],
         ...,
         [0.3876],
         [0.4536],
         [0.3442]]),
 torch.Size([3000, 1]))

In [31]:
loss = -(1/m)*torch.sum(y*torch.log(sigma) + (1-y)*torch.log(1-sigma))

In [32]:
loss,loss.shape

(tensor(0.7962), torch.Size([]))

In [33]:
# loss = -(1/m)*sum(y*torch.log(sigma) + (1-y)*torch.log(1-sigma))
# python中的sum和torch.sum 当样本量非常大时使用torch中的函数对于张量来说运行更快

In [41]:
import time
m = 3*pow(10,6)
torch.random.manual_seed(420) 
X = torch.rand((m,4),dtype=torch.float32)
w = torch.rand((4,1),dtype=torch.float32)
y = torch.randint(low=0,high=2,size=(m,1),dtype=torch.float32)
zhat = torch.mm(X,w)
sigma = torch.sigmoid(zhat)

In [42]:
start = time.time() #捕获现在的时间
loss1 = -(1/m)*sum(y*torch.log(sigma) + (1-y)*torch.log(1-sigma))
now = time.time() #以秒计时
print(now - start)

30.35755205154419


In [43]:
start = time.time()
loss2 = -(1/m)*torch.sum(y*torch.log(sigma) + (1-y)*torch.log(1-sigma))
now = time.time()
print(now - start)

0.06929492950439453


In [44]:
# torch.nn 内置两个类计算二分类交叉熵损失
# BCEWithLOgitsLoss
# BCELoss
# 同样目的的两个类，输入不同，精度不同
# BCEWithLOgitsLoss的精度更高
# 监控准确率用BCELoss，其输入是sigma，避免重复计算

In [45]:
import torch.nn as nn

In [46]:
criterion = nn.BCELoss(reduction) #实例化
loss = criterion(sigma,y)

In [47]:
loss

tensor(0.8685)

In [49]:
criterion2 = nn.BCEWithLogitsLoss()

In [50]:
loss = criterion2(zhat,y)

In [51]:
loss

tensor(0.8685)

In [53]:
# none结果为矩阵，sum为加和，mean为平均
criterion = nn.BCELoss(reduction="mean") #实例化
loss1 = criterion(sigma,y)
criterion2 = nn.BCEWithLogitsLoss(reduction="sum")
loss2 = criterion2(zhat,y)
criterion3 = nn.BCEWithLogitsLoss(reduction="none")
loss3 = criterion3(zhat,y)
loss1,loss2,loss3

(tensor(0.8685),
 tensor(2605616.5000),
 tensor([[1.3102],
         [0.3155],
         [0.4247],
         ...,
         [0.1727],
         [0.1716],
         [0.1673]]))

In [54]:
# 多分类交叉熵

In [55]:
import torch
import torch.nn as nn

In [56]:
m = 3*pow(10,3)

In [67]:
torch.random.manual_seed(420)
X = torch.rand((m,4),dtype=torch.float32)
w = torch.rand((4,3),dtype=torch.float32)
y = torch.randint(low=0,high=3,size=(m,),dtype=torch.float32)

In [58]:
zhat = torch.mm(X,w)

In [59]:
# sigma = torch.softmax(zhat)

In [60]:
# logsoftmax 包含 log + softmax 功能,需要输入：zhat

In [61]:
# NLLLoss()

In [63]:
logsm = nn.LogSoftmax(dim=1) #对哪个维度进行softmax计算

In [64]:
logsigma = logsm(zhat)

In [65]:
logsigma,logsigma.shape

(tensor([[-1.1139, -0.8802, -1.3585],
         [-1.0558, -0.8982, -1.4075],
         [-1.0920, -1.0626, -1.1430],
         ...,
         [-1.0519, -0.9180, -1.3805],
         [-1.0945, -1.1219, -1.0798],
         [-1.0276, -0.8891, -1.4649]]),
 torch.Size([3000, 3]))

In [66]:
criterion = nn.NLLLoss()

In [71]:
loss = criterion(logsigma,y.long()) #需要标签独热编码矩阵即系数矩阵，使用.long转换为整形
loss

tensor(1.1147)

In [69]:
# 调用包装好的函数CrossEntropyLoss
criterion = nn.CrossEntropyLoss() #jupyter快捷键tab有提示

In [72]:
loss = criterion(zhat,y.long())
loss

tensor(1.1147)